## Clasificador y generador de dígitos utilizando naive bayes
### Pablo Gonzalez Baron

---

### Importar librerias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar datos MNIST

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

from matplotlib import pyplot
pyplot.subplot(331)
pyplot.imshow(x_train[0], cmap=pyplot.get_cmap('gray'))
pyplot.show()

### Transformar datos

In [ ]:
x_train = x_train.reshape(60000,784)
x_test = x_test.reshape(10000,784)
data = np.zeros([70000,784])
data[:60000,:] = x_train; data[60000:,:] = x_test
label = np.zeros(70000)
label[:60000] = y_train; label[60000:] = y_test

### Construcción del modelo Naive Bayes
A continuación se construye la función del modelo Naive Bayes que entrena el modelo con los datos de entrenamiento, lo evalua con los datos de test y luego genera una matriz de confusión, junto a la tasa de error de clasificación de cada digito individual.

In [ ]:
def naive_bayes(data,label):
    n_s,n_f = data.shape          # Obtenemos el shape (sample, features) de nuestros datos
    classes = np.unique(label)    # Obtenemos el nombre de las clases en las etiquetas de nuestros datos
    n_c = len(classes)            # Numero de clases
    total_data = np.zeros([n_s,n_f+1]) # Creamos una matriz de ceros con dimensiones (samples, feature+1) para guardar, como lo dicen las dimensiones, los samples y las features.
    total_data[:,:-1] = data
    total_data[:,-1] = label           # Guardamos las etiquetas en esta matriz previamente creada
    np.random.shuffle(total_data)
    X_train = total_data[:60000,:]
    np.random.shuffle(X_train)
    X_test = total_data[60000:,:]
    np.random.shuffle(X_test)
    X_test_c = X_test[:,:-1]             # Obtener samples y feature de la data de test
    X_test_l = X_test[:,-1]              # Obtener labels de la data de test
    mean_v = np.zeros([n_c,n_f])       # Crear una matriz de ceros que será usada para guardar la media de las features y clases
    var_v = np.zeros([n_c,n_f])        # Crear una matriz de ceros que será usada para guardar la varianza de las features y clases
    c_prob = []                        # Probabilidades del modelo
    confusion_matrix = np.zeros([n_c,n_c]) # Crear matriz de confusion de tamaño (classes*classes)
    d_acc = []                         # Accuracy de clasificación de cada dígito
    
    for c in classes:
        X_train_c = X_train[X_train[:,-1] == c]
        X_train_c = X_train_c[:,:-1]
        c_prob.append(len(X_train_c)/len(X_train))
        mean_v[int(c),:] = X_train_c.mean(axis=0)
        var_v[int(c),:] = X_train_c.var(axis=0)
    
    var_v = var_v + 1000
    count = 0             
    
    for i in range(X_test.shape[0]):
        lists=[]   # Lista para almacenar la probabilidad de todas las clases para la observación i.
        for j in range(n_c):
            numerator = np.exp(-((X_test_c[i]-mean_v[j])**2)/(2*var_v[j])) 
            denominator = np.sqrt(2*np.pi*(var_v[j]))
            ratio=np.sum(np.log(numerator/denominator)) # Probabilidad de que la feature i sea de la clase j
            lists.append(ratio) # Añadir probabilidad de que la feature i sea de la clase j
        
        pred = lists.index(max(lists)) # Tomar el y predicho para la clase que tiene el valor maximo de probabilidad en el vector de predicción
        if pred == X_test_l[i]: 
            count = count+1 # Si el valor predicho es igual al valor real, se incrementa el conteo
            confusion_matrix[int(X_test_l[i])][int(X_test_l[i])] = confusion_matrix[int(X_test_l[i])][int(X_test_l[i])]+1
            # Se añaden los valores correspondientes la matriz de confusión
        else:
            for k in range(n_c):
                if pred == k:
                    confusion_matrix[int(X_test_l[k])][int(X_test_l[i])] = confusion_matrix[int(X_test_l[k])][int(X_test_l[i])]+1
                    # Se añaden los valores correspondientes la matriz de confusión
    for l in classes:
        check = X_test[X_test[:,-1] == l]
        a = (confusion_matrix[int(l)][int(l)])/check.shape[0] # Calcular el accuracy de cada digito
        d_acc.append(a)   # Añadir accuracy individual de cada digito
    

    o_acc = count/X_test.shape[0]
    return(d_acc, o_acc, confusion_matrix, mean_v, var_v)

### Predicciones con el modelo Naive Bayes

A continuación vamos a usar la función que creamos anteriormente para ajustar nuestro modelo Naive Bayes a nuestro conjunto de entrenamiento, para luego evaluar nuestro modelo usando nuestro conjunto de test.

In [ ]:
digit_accuracy, overall_accuracy, matrix, mean_v, var_v = naive_bayes(data,label)

### Graficar la media y la varianza de cada dígito

In [ ]:
# mean of each digit in 28*28 form
plt.figure(figsize=(16,8))
for i in range(mean_v.shape[0]):
    plt.subplot(2, 5, i+1)
    img = mean_v[i].reshape(28,28)
    plt.imshow(img, cmap="Greys")
    plt.xlabel('Image of digit '+str(i)+' Mean',fontsize = 12)

In [ ]:
# variance of each digit in 28*28 form
plt.figure(figsize=(16,8))
for i in range(var_v.shape[0]):
    plt.subplot(2,5, i+1)
    img = var_v[i].reshape(28,28)
    plt.imshow(img, cmap="Greys")
    plt.xlabel('Image of digit '+str(i)+' Variance',fontsize = 12)

### Desempeño de generalizacion del modelo
En esta ocasión utilizamos el accuracy de clasificación de cada dígito y la matriz de confusión para medir la capacidad del modelo para generalizar.

In [ ]:
digit = np.unique(label).astype('int64')
naive_df = pd.DataFrame(list(zip(digit, digit_accuracy)), columns = ['Digito','Accuracy'])
naive_df.set_index('Digito', inplace=True)
print('Accuracy de clasificación para cada dígito:')
naive_df

In [ ]:
#Print Confusion Matrix
import seaborn as sns
plt.figure(figsize=(14,7))
sns.heatmap(matrix, annot=True, cmap="YlGnBu")
plt.xlabel('True class value')
plt.ylabel('Predicted class value');